<a href="https://colab.research.google.com/github/hibahkhan2022-rgb/Edge-AI-Product-Detection-Inferencing-Workshop/blob/main/EdgeAIProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount drive to access data files
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [7]:
import os

# this is where the classification folders are under
DATA_DIR = "/content/gdrive/MyDrive/skincare-edge-ai/data/images/train"

assert os.path.isdir(DATA_DIR), f"Missing: {DATA_DIR}"
print("DATA_DIR contents:", os.listdir(DATA_DIR))

# ensure class folders are present
for cls in ["makeup", "skincare", "scents"]:
    p = os.path.join(DATA_DIR, cls)
    print(cls, "exists:", os.path.isdir(p), "| num items:", len(os.listdir(p)) if os.path.isdir(p) else "NA")

DATA_DIR contents: ['makeup', 'scents', 'skincare']
makeup exists: True | num items: 120
skincare exists: True | num items: 99
scents exists: True | num items: 115


In [8]:
import random, numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights

#initalize seeds
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

#can run on either cpu/gpu; for further inferencing, use GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cpu


In [9]:
#Using MobileNetV3 lightweight model; standard image resolution
IMG_SIZE = 224

#tranform properties for training dataset adjust for images, shadows, alignment, etc.
train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

#tranform properties for validation dataset adjust for images, shadows, alignment, etc.
val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

In [10]:
from collections import Counter

#checks counts per class, make sure distribution is even
full_ds = datasets.ImageFolder(DATA_DIR, transform=train_tfms)
class_names = full_ds.classes
num_classes = len(class_names)

print("Classes:", class_names)
print("Total images:", len(full_ds))

counts = Counter(full_ds.targets)
print("Counts per class:")
for i, name in enumerate(class_names):
    print(f"  {name:10s}: {counts.get(i, 0)}")

Classes: ['makeup', 'scents', 'skincare']
Total images: 333
Counts per class:
  makeup    : 119
  scents    : 115
  skincare  : 99


In [11]:
#defines batch parameters
BATCH_SIZE = 32
NUM_WORKERS = 2
VAL_FRAC = 0.2

#Split dataset 80/20
val_size = int(VAL_FRAC * len(full_ds))
train_size = len(full_ds) - val_size

train_ds, val_ds = random_split(
    full_ds,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(SEED)
)

#change val transform
val_ds.dataset.transform = val_tfms

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)

print("Train size:", len(train_ds), "| Val size:", len(val_ds))

Train size: 267 | Val size: 66


In [12]:
#load MobileNetV3 model
weights = MobileNet_V3_Small_Weights.DEFAULT
model = mobilenet_v3_small(weights=weights)

in_features = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(in_features, num_classes)

model = model.to(device)
print("New head:", model.classifier[-1])

Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


100%|██████████| 9.83M/9.83M [00:00<00:00, 42.2MB/s]

New head: Linear(in_features=1024, out_features=3, bias=True)


In [13]:
#Use Cross-Entropy Loss and AdamW optimizer
LR = 3e-4
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

In [14]:
import os

EPOCHS = 10
os.makedirs("/content/models", exist_ok=True)
best_path = "/content/models/mnv3_best.pth"

def batch_acc(logits, y):
    return (logits.argmax(1) == y).float().mean().item()

def train_one_epoch():
    model.train()
    total_loss, total_acc, n = 0.0, 0.0, 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        bs = x.size(0)
        total_loss += loss.item() * bs
        total_acc  += batch_acc(logits, y) * bs
        n += bs
    return total_loss / n, total_acc / n

@torch.no_grad()
def eval_one_epoch():
    model.eval()
    total_loss, total_acc, n = 0.0, 0.0, 0
    for x, y in val_loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        loss = criterion(logits, y)

        bs = x.size(0)
        total_loss += loss.item() * bs
        total_acc  += batch_acc(logits, y) * bs
        n += bs
    return total_loss / n, total_acc / n

best_val_acc = 0.0
for epoch in range(1, EPOCHS + 1):
    tr_loss, tr_acc = train_one_epoch()
    va_loss, va_acc = eval_one_epoch()
    print(f"Epoch {epoch:02d}/{EPOCHS} | train {tr_loss:.4f} {tr_acc:.4f} | val {va_loss:.4f} {va_acc:.4f}")

    if va_acc > best_val_acc:
        best_val_acc = va_acc
        torch.save({"model_state": model.state_dict(),
                    "classes": class_names,
                    "img_size": IMG_SIZE}, best_path)
        print(f"saved best (val acc {best_val_acc:.4f}) -> {best_path}")

print("Best val acc:", best_val_acc)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 01/10 | train 0.9299 0.6142 | val 0.6941 0.7424
  ✅ saved best (val acc 0.7424) -> /content/models/mnv3_best.pth
Epoch 02/10 | train 0.4765 0.8801 | val 0.5590 0.8030
  ✅ saved best (val acc 0.8030) -> /content/models/mnv3_best.pth
Epoch 03/10 | train 0.2572 0.9551 | val 0.5163 0.8182
  ✅ saved best (val acc 0.8182) -> /content/models/mnv3_best.pth
Epoch 04/10 | train 0.1487 0.9700 | val 0.4797 0.8333
  ✅ saved best (val acc 0.8333) -> /content/models/mnv3_best.pth
Epoch 05/10 | train 0.0784 0.9850 | val 0.5247 0.7727
Epoch 06/10 | train 0.0380 0.9963 | val 0.5838 0.7576
Epoch 07/10 | train 0.0202 1.0000 | val 0.6402 0.7727
Epoch 08/10 | train 0.0188 0.9963 | val 0.5577 0.8182
Epoch 09/10 | train 0.0074 1.0000 | val 0.5216 0.8182
Epoch 10/10 | train 0.0060 1.0000 | val 0.5206 0.8333
Best val acc: 0.8333333333333334


In [17]:
import torch
from torchvision.models import mobilenet_v3_small
from torch import nn

#load from the best epoch
device = "cuda" if torch.cuda.is_available() else "cpu"

ckpt_path = "/content/models/mnv3_best.pth"
ckpt = torch.load(ckpt_path, map_location=device)

class_names = ckpt["classes"]
IMG_SIZE = ckpt["img_size"]

model = mobilenet_v3_small(weights=None)
in_features = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(in_features, len(class_names))
model.load_state_dict(ckpt["model_state"])
model = model.to(device).eval()

print("Loaded best model. Classes:", class_names)

Loaded best model. Classes: ['makeup', 'scents', 'skincare']


In [18]:
import numpy as np
import torch
from sklearn.metrics import confusion_matrix, classification_report

#detail confusion matrix and classification report
all_preds, all_y = [], []

model.eval()
with torch.no_grad():
    for x, y in val_loader:
        x = x.to(device)
        logits = model(x)
        preds = logits.argmax(1).cpu().numpy()
        all_preds.extend(preds)
        all_y.extend(y.numpy())

cm = confusion_matrix(all_y, all_preds)
print("Confusion matrix:\n", cm)

print("\nClassification report:")
print(classification_report(all_y, all_preds, target_names=class_names))


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Confusion matrix:
 [[25  1  4]
 [ 0 11  5]
 [ 1  0 19]]

Classification report:
              precision    recall  f1-score   support

      makeup       0.96      0.83      0.89        30
      scents       0.92      0.69      0.79        16
    skincare       0.68      0.95      0.79        20

    accuracy                           0.83        66
   macro avg       0.85      0.82      0.82        66
weighted avg       0.86      0.83      0.84        66

